## Learning Machine Learning 2018
## Universidad del Rosario

##### Integrantes
##### German
##### Hugo
##### Arturo
##### Jimmy



In [1]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Fri Jul 27 16:46:10 2018

@author: felipe
"""

import os
import glob
import h5py
import shutil
import imgaug as aug
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mimg
import imgaug.augmenters as iaa
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
from pathlib import Path
from skimage.io import imread
from skimage.transform import resize
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, SeparableConv2D
from keras.layers import GlobalMaxPooling2D, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
import cv2
from keras import backend as K

/Users/jvivas/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import tensorflow as tf

# Set the seed for hash based operations in python
os.environ['PYTHONHASHSEED'] = '0'

# Set the numpy seed
np.random.seed(111)

# Disable multi-threading in tensorflow ops
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

# Set the random seed in tensorflow at graph level
tf.set_random_seed(111)

# Define a tensorflow session with above session configs
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)

# Set the session in keras
K.set_session(sess)

# Make the augmentation sequence deterministic
aug.seed(111)

### Data Extraction

In [3]:
# Define path to the data directory
data_dir = Path('/Users/jvivas/Documents/Kaggle/XRay Pneumonia/data/chest_xray')

# Path to train directory (Fancy pathlib...no more os.path!!)
train_dir = data_dir / 'train'

# Path to validation directory
val_dir = data_dir / 'val'

# Path to test directory
test_dir = data_dir / 'test'

normal_casos_dir = train_dir / 'NORMAL'
pneumonia_casos_dir = train_dir / 'PNEUMONIA'

# put the images in a list
normal_casos_lista = normal_casos_dir.glob('*.jpeg')
pneumonia_casos_lista = pneumonia_casos_dir.glob('*.jpeg')

### Image Categorization

In [4]:
train_data1 = []

img_size = [225, 225]

# agregamos la label a cada foto
for img in normal_casos_lista:
    train_data1.append((img,0))

for img in pneumonia_casos_lista:
    train_data1.append((img,1))

### Shuffeling Images

In [5]:
# Shuffle
s = np.arange(len(train_data1))
np.random.shuffle(s)

### Processing Images Train dataset

In [6]:
train_data = [0]*len(train_data1)
for i in range(len(train_data)):
    train_data[i] = train_data1[s[i]]


train_img_data =[]
train_labels = []
for i in range(len(train_data)):
    img = cv2.imread(str(train_data[i][0]))
    if img.shape[2]==3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (img_size[0], img_size[1]))
    post_img= cv2.equalizeHist(img)
    label = train_data[i][1]
    #plt.imshow(post_img, cmap='gray')
    post_img = post_img.reshape(225,225,1)
    train_img_data.append(post_img)
    train_labels.append(label)

### Normalization

In [7]:
X_train = np.array(train_img_data) / 255
Y_train = np.array(train_labels)

### Build CNN

In [8]:
class LeNet:
    @staticmethod
    def build(width, height, ker_1 = 10,  ker_2 =20 , drop = 0.05):
        # initialize the model
        ker_1_size=(7,7)
        ker_2_size=(7,7)

        model = Sequential()
        inputShape = (height, width,1)

        # first set of CONV => RELU => POOL layers
        model.add(Conv2D(ker_1, ker_1_size, input_shape=inputShape, padding="same", kernel_initializer = 'normal'))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # second set of CONV => RELU => POOL layers
        model.add(Conv2D(ker_2, ker_2_size, padding="same", kernel_initializer = 'normal'))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),)
        model.add(Dropout(drop))
        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(50))
        model.add(Activation("relu"))

        # softmax classifier
        model.add(Dense(1))
        model.add(Activation("softmax"))

        # return the constructed network architecture
        return model

In [9]:
# Building CNN Model
model = LeNet.build(img_size[0], img_size[1])

In [10]:
# Compiling model 
chkpt = ModelCheckpoint(filepath='best_model_todate', save_best_only=True, save_weights_only=True)
es = EarlyStopping(patience=5)
model.compile(loss = 'binary_crossentropy' 
              , optimizer = 'adagrad' 
              , metrics=['accuracy'])

In [11]:
history = model.fit(X_train, Y_train, batch_size = 16, epochs = 20,
                    validation_split = 0.2, verbose =1
                    ,callbacks=[es, chkpt])

Train on 4172 samples, validate on 1044 samples
Epoch 1/20
4172/4172 [==============================] - 800s 192ms/step - loss: 4.1690 - acc: 0.7385 - val_loss: 3.8176 - val_acc: 0.7605
Epoch 2/20
4172/4172 [==============================] - 600s 144ms/step - loss: 4.1690 - acc: 0.7385 - val_loss: 3.8176 - val_acc: 0.7605
Epoch 3/20
4172/4172 [==============================] - 599s 144ms/step - loss: 4.1690 - acc: 0.7385 - val_loss: 3.8176 - val_acc: 0.7605
Epoch 4/20
4172/4172 [==============================] - 597s 143ms/step - loss: 4.1690 - acc: 0.7385 - val_loss: 3.8176 - val_acc: 0.7605
Epoch 5/20
4172/4172 [==============================] - 598s 143ms/step - loss: 4.1690 - acc: 0.7385 - val_loss: 3.8176 - val_acc: 0.7605
Epoch 6/20
4172/4172 [==============================] - 598s 143ms/step - loss: 4.1690 - acc: 0.7385 - val_loss: 3.8176 - val_acc: 0.7605


In [ ]:
# Save weights
model.load_weights('/Users/jvivas/Documents/Kaggle/XRay Pneumonia/best_model.hdf5')

In [13]:
normal_casos_dir = test_dir / 'NORMAL'
pneumonia_casos_dir = test_dir / 'PNEUMONIA'

# put the images in a list
normal_casos_lista = normal_casos_dir.glob('*.jpeg')
pneumonia_casos_lista = pneumonia_casos_dir.glob('*.jpeg')

test_data = []

# agregamos la label a cada foto
for img in normal_casos_lista:
    test_data.append((img,0))

for img in pneumonia_casos_lista:
    test_data.append((img,1))

test_img_data =[]
test_labels = []
for i in range(len(test_data)):
    img = cv2.imread(str(test_data[i][0]))
    if img.shape[2]==3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (img_size[0], img_size[1]))
    post_img= cv2.equalizeHist(img)
    label = train_data[i][1]
    #plt.imshow(post_img, cmap='gray')
    post_img = post_img.reshape(225,225,1)
    test_img_data.append(post_img)
    test_labels.append(label)

# Normalization
X_test = np.array(test_img_data) / 255
Y_test = np.array(test_labels)

In [14]:
# Evaluation on test dataset
test_loss, test_score = model.evaluate(X_test, Y_test, batch_size=16)
print("Loss on test set: ", test_loss)
print("Accuracy on test set: ", test_score)

624/624 [==============================] - 35s 56ms/step
Loss on test set:  4.266631966982134
Accuracy on test set:  0.7323717948717948


In [ ]:
# Get predictions
preds = model.predict(Y_test, batch_size=16)
preds = np.argmax(preds, axis=-1)

# Original labels
orig_test_labels = np.argmax(Y_test, axis=-1)

print(orig_test_labels.shape)
print(preds.shape)

When a particular problem includes an imbalanced dataset, then accuracy isn't a good  metric to look for. For example, if your dataset contains 95 negatives and 5 positives, having a model with 95% accuracy doesn't make sense at all. The classifier might label every example as negative and still achieve 95% accuracy. Hence,  we need to look for alternative metrics. **Precision** and **Recall** are really good metrics for such kind of problems. 

We will get the confusion matrix from our predictions and see what is the recall and precision of our model.

In [ ]:
# Get the confusion matrix
cm  = confusion_matrix(orig_test_labels, preds)
plt.figure()
plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True, alpha=0.7,cmap=plt.cm.Blues)
plt.xticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
plt.yticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
plt.show()

In [ ]:
# Calculate Precision and Recall
tn, fp, fn, tp = cm.ravel()

precision = tp/(tp+fp)
recall = tp/(tp+fn)

print("Recall of the model is {:.2f}".format(recall))
print("Precision of the model is {:.2f}".format(precision))

Nice!!! So, our model has a 98% recall. In such problems, a good recall value is expected. But if you notice, the precision is only 80%. This is one thing to notice. Precision and Recall follows a trade-off, and you need to find a point where your recall, as well as your precision, is more than good but both can't increase simultaneously. 

That's it folks!! I hope you enjoyed this kernel. Happy Kaggling!!